In [1]:
#pip install bs4

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# modify html to text and read
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source)
post_table = soup.find("table", {"class": "wikitable"})

headers = {0: 'Postcode', 1: 'Borough', 2: 'Neighbourhood'}

rows = post_table.find_all('tr')

table_data = []

#first row include head of table, create our table
for count in range(1,len(rows)):
    cells = rows[count].find_all("td")
    item = {}
    for index in headers:
        item[headers[index]] = cells[index].text
    table_data.append(item)   

#table - to dataframe
df = pd.DataFrame(table_data)
df.head()
df.shape

(287, 3)

In [3]:
# delete rows where Borough  = 'Not assigned'
df = df.loc[df['Borough'] != 'Not assigned']

# delete symbols \n in Neighbourhood column
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n','')

# grouped all rows and aggregate Neighbourhood separated by ','
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(', '.join).reset_index()

# replace 'Not assigned' in Neighbourhood on Borough
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']      
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
#geocoder not worked, I leave in Belarus and use another proxy to download data and joined dataframes
df2 = pd.read_csv('data/Geospatial_Coordinates.csv')
df2 = df2.rename(columns={'Postal Code': 'Postcode'})

df = df.merge(df2, on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
